# Problem Formulation

## Problem Description:
This project aims to develop a machine learning model that predicts future sales
and demand by utilising historical sales data and external factors, including
product details, promotions, seasonality, holidays, and economic indicators. The
goal is to analyse historical patterns and generate reliable forecasts that help
businesses make data-driven decisions to reduce costs, increase efficiency, and
improve customer satisfaction by predicting the daily sales for the next 28 days.

## Objectives

● Collect and preprocess historical sales and demand data.

● Identify key features that influence sales trends.

● Build, train, and optimise forecasting models to predict future sales and
demand.

● Deploy the best-performing model to generate forecasts in real-time or in
batches.
## Data source:
Hierarchical sales data from Walmart, the world’s largest company by revenue in the US.



# Code setup

## Important libraries

In [1]:
import pandas as pd
import numpy as np

## data reading

In [2]:
sales_validation=pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')
sales_validation.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1


In [3]:
#cal=pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
#cal.head(3)

In [4]:
sell_price = pd.read_csv("/kaggle/input/m5-forecasting-accuracy/sell_prices.csv")
sell_price.head(3)

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26


In [5]:
temp_data = pd.read_csv("/kaggle/input/depi-dataset/temp_data.csv")

# EDA

In [6]:
#cal.info()

In [7]:
temp_data.head(3)

,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap
0,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No event,No event,No event,No event,0
1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No event,No event,No event,No event,0
2,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,1,1,2011,No event,No event,No event,No event,0


In [8]:
#final_data.head()

In [9]:
#cal.head()

# Schema formating

## sales_validation file

In [10]:
'''
data = sales_validation.melt(
    id_vars=["id","item_id", "dept_id","cat_id","store_id","state_id"],  # columns to keep
    var_name="d",                          # new column name for day labels (d_1, d_2, ...)
    value_name="sales"                     # new column name for sales values
)
'''


'\ndata = sales_validation.melt(\n    id_vars=["id","item_id", "dept_id","cat_id","store_id","state_id"],  # columns to keep\n    var_name="d",                          # new column name for day labels (d_1, d_2, ...)\n    value_name="sales"                     # new column name for sales values\n)\n'

In [11]:
#data.drop(columns=['id'],inplace=True)

## calendar file

In [12]:
#cal["date"]= pd.to_datetime(cal["date"])

In [13]:
'''
cal["event_name_1"]= cal["event_name_1"].fillna("No event")
cal["event_type_1"]= cal["event_type_1"].fillna("No event")
cal["event_name_2"]= cal["event_name_2"].fillna("No event")
cal["event_type_2"]= cal["event_type_2"].fillna("No event")
'''

'\ncal["event_name_1"]= cal["event_name_1"].fillna("No event")\ncal["event_type_1"]= cal["event_type_1"].fillna("No event")\ncal["event_name_2"]= cal["event_name_2"].fillna("No event")\ncal["event_type_2"]= cal["event_type_2"].fillna("No event")\n'

In [14]:
'''
# Merge data and cal dataframes on the 'd' column
merged_data = pd.merge(data, cal, on='d', how='left')

conditions = [
    merged_data["state_id"] == "CA",
    merged_data["state_id"] == "TX",
    merged_data["state_id"] == "WI"
]
choices= [
    merged_data["snap_CA"],
    merged_data["snap_TX"],
    merged_data["snap_WI"]
]
merged_data["snap"]= np.select(conditions, choices)

merged_data.drop(columns=['snap_CA','snap_TX','snap_WI'],inplace=True)
'''

'\n# Merge data and cal dataframes on the \'d\' column\nmerged_data = pd.merge(data, cal, on=\'d\', how=\'left\')\n\nconditions = [\n    merged_data["state_id"] == "CA",\n    merged_data["state_id"] == "TX",\n    merged_data["state_id"] == "WI"\n]\nchoices= [\n    merged_data["snap_CA"],\n    merged_data["snap_TX"],\n    merged_data["snap_WI"]\n]\nmerged_data["snap"]= np.select(conditions, choices)\n\nmerged_data.drop(columns=[\'snap_CA\',\'snap_TX\',\'snap_WI\'],inplace=True)\n'

## sell_price file


In [15]:
final_data = temp_data.merge(
        sell_price,
        on=["store_id", "item_id", "wm_yr_wk"],
        how="left",
        validate="m:1"  # many sales rows per unique price row is expected
    )

In [16]:
final_data["price_in_dollars"]=final_data["sell_price"]*final_data["sales"]

In [17]:
final_data.to_csv("/kaggle/working/data.csv")